# Import Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

# Load Dataset

In [ ]:
traindf = pd.read_csv('normalised_df.csv')

In [ ]:
X = traindf.drop('emission', axis=1)

In [ ]:
y = traindf['emission']

# Split the data into training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 2)

# Random Forest Model (Regression)

In [ ]:
RFR_model = RandomForestRegressor()

In [ ]:
min_rmse = float('inf')
depth = 0
for i in range(6, 21, 1):
    RFR_model = RandomForestRegressor(max_depth= i)
    print("depth : "+ str(i))
    RFR_model.fit(X_train, y_train)
    y_pred = RFR_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(rmse)
    if(rmse < min_rmse):
        min_rmse = rmse
        depth = i

print(f"Min RMSE = {str(min_rmse)}, depth = {str(depth)}")


In [ ]:
RFR_model = RandomForestRegressor(max_depth= 21, n_estimators= 250)
RFR_model.fit(X_train, y_train)
y_pred = RFR_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

# Catboost Regression

In [ ]:
CBR_model = CatBoostRegressor()

In [78]:
CBR_model = CatBoostRegressor(
    iterations=5000,  # You can adjust the number of iterations
    early_stopping_rounds=50,  # Stop if performance doesn't improve for 50 rounds
    eval_metric='RMSE'  # Evaluation metric
)

# Train the model
CBR_model.fit(X_train, y_train, eval_set=(X_test, y_test),verbose= 100)

300:	learn: 59.1658146	test: 57.8013072	best: 57.8013072 (300)	total: 9.29s	remaining: 2m 25s


KeyboardInterrupt: 

In [ ]:
y_pred_cbr = CBR_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
rmse

# Test DataSet

In [ ]:
features = traindf.columns

In [ ]:
features = features.delete(-1)

In [ ]:
features

In [ ]:
testdf = pd.read_csv('test.csv', index_col= 'ID_LAT_LON_YEAR_WEEK')

In [ ]:
testdf.shape

In [ ]:
test = testdf[features]

In [ ]:
test.shape

In [ ]:
imputer = KNNImputer(n_neighbors=3)  # Choose the number of nearest neighbors
test_imputed_data = imputer.fit_transform(test)
test_imputed_df = pd.DataFrame(test_imputed_data, columns=test.columns)

In [ ]:
test_imputed_df.head()

In [ ]:
test_imputed_df['year'] = test_imputed_df['year'].astype('Int64')
test_imputed_df['week_no'] = test_imputed_df['week_no'].astype('Int64')

In [ ]:
test_imputed_df.head()

In [ ]:
test_imputed_df.to_csv('test_imputed_df.csv', index = True)

In [ ]:
# import folium
# map_center = [test_imputed_df['latitude'].mean(), test_imputed_df['longitude'].mean()]
# m = folium.Map(location=map_center, zoom_start=10)

# # Add markers for each latitude and longitude
# for index, row in test_imputed_df.iterrows():
#     folium.Marker([row['latitude'], row['longitude']]).add_to(m)

# m  # Display the map

In [ ]:
for col in test_imputed_df.columns:
    if col == "emission" or col == "latitude" or col == "longitude" or col == "year" or col == "week_no":
        print()
    else:
        print(col)
        test_imputed_df[col] = (test_imputed_df[col] - test_imputed_df[col].mean())/test_imputed_df[col].std()

In [ ]:
test_imputed_df

In [ ]:
y_predictions = RFR_model.predict(test_imputed_df)

In [ ]:
y_predictions

In [ ]:
submission = pd.DataFrame({'ID_LAT_LON_YEAR_WEEK': testdf.index, 'emission':y_predictions,})

In [ ]:
submission

In [ ]:
submission.to_csv('submission1.csv', index= False)